In [172]:
# Importing required libraries for optimization
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [173]:
def least_squares(A, b):
    '''
    Solves the least squares problem Ax = b
    Args:
        A: co-efficients matrix
        b: constants vector
    Returns:
        x: solution vector
    '''

    return np.linalg.inv(A.T @ A) @ A.T @ b

In [174]:
# Finite difference methods:
def forward_diff(f, x, h=1E-08):
    '''
    Solves the forward difference problem
    Args:
        f: function
        x: vector of variables
        h: step size
    Returns:
        delta_f: solution vector
    '''

    delta_f = np.zeros(x.shape) # Initialize delta_f
    for i in range(x.shape[0]):
        x_forw = np.array(x) # Make a copy of x
        x_forw[i] += h   # Increment x_forw[i] by h
        delta_f[i] = (f(x_forw) - f(x)) / h # Calculate the forward difference

    return delta_f


def central_diff(f, x, h=1E-08):
    '''
    Solves the central difference problem
    Args:
        f: function
        x: vector of variables
        h: step size
    Returns:
        delta_f: solution vector
    '''

    delta_f = np.zeros(x.shape) # Initialize delta_f
    for i in range(x.shape[0]):
        x_forw = np.array(x) # Make a copy of x
        x_back = np.array(x) # Make a copy of x
        x_forw[i] += h   # Increment x_forw[i] by h
        x_back[i] -= h   # Decrement x_back[i] by h
        delta_f [i]= (f(x_forw) - f(x_back)) / (2*h) # Calculate the central difference

    return delta_f

In [175]:
# Converting constrained optimization problem to unconstrained optimization problem:
# Using penalty method:
def constrained_to_unconstrained(f, con, x, p=100):
    '''
    Converts a constrained optimization problem to an unconstrained optimization problem
    Args:
        f: function
        con: constraint function
        x: vector of variables
        p: penalty parameter
    Returns:
        f_uncon: unconstrained function
    '''
    
    # Objective function
    def f_unconstrained(x):
        g, h = con(x) # Calculate the constraint function
        con_sum = 0 # Initialize the constraint sum
        for i in range(g.shape[0]):
            con_sum += max(0, g[i])**2 # Add the constraint sum
        for i in range(h.shape[0]):
            con_sum += h[i]**2  # Add the constraint sum
        return f(x) + p*con_sum # Return the unconstrained objective function
        
    return f_unconstrained

In [176]:
# Unconstrained optimization:
# 1st order methods:

def steepest_descent(f, x, delta_f, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False):
    '''
    Optimizes the unconstrained problem using the steepest descent method
    Args:
        f: objective function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''

    convergence = False
    iter = 0
    while not convergence:
        s = -delta_f(f, x) # Calculate the search direction
        s = s / np.linalg.norm(s) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the step size
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter
        
        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3
            
        x = x_new # Update x

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new


def conjugate_gradient(f, x, delta_f, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False):
    '''
    Optimizes the unconstrained problem using the conjugate gradient method
    Args:
        f: objective function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''

    s = -delta_f(f, x) # Calculate the initial search direction
    s = s / np.linalg.norm(s) # Normalize the initial search direction
    alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the initial step size
    x_new = x + alpha*s # Calculate the initial x update

    convergence = False
    iter = 0
    while not convergence:
        if iter % x.shape[0] == 0:  # Check if the iteration counter is a multiple of the number of design variables
            s = -delta_f(f, x_new) # Calculate the search direction
        else:
            s = -delta_f(f, x_new) + (np.linalg.norm(delta_f(f, x_new))**2 / np.linalg.norm(delta_f(f, x))**2) * s # Calculate the search direction

        s = s / np.linalg.norm(s) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x_new + alpha*s), 0, 1) # Calculate the step size
        x = x_new # Update x
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter

        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3
        
    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new


# Hessian update rules:
def BFGS_update(B, delta_x, delta_delta):
    '''
    Calculates the BFGS update matrix
    Args:
        B: initial Hessian estimate
        delta_x: design variable update
        delta_delta: objective function update
    Returns:
        deta_B: update to the Hessian estimate
    '''

    return (1 + delta_delta.T@B@delta_delta / (delta_delta.T@delta_x)) * (delta_x@delta_x.T) / (delta_x.T@delta_delta) - \
        (delta_x@(delta_delta.T@B) + (delta_delta.T@B).T@delta_x.T) / (delta_x.T@delta_delta)

def DFP_update(B, delta_x, delta_delta):
    '''
    Calculates the DFP update matrix
    Args:
        B: initial Hessian estimate
        delta_x: design variable update
        delta_delta: objective function update
    Returns:
        deta_B: update to the Hessian estimate
    '''

    return delta_x@delta_x.T / (delta_x.T@delta_delta) - \
        (B@delta_delta)@(B@delta_delta).T / (delta_delta.T@B@delta_delta)


# Quasi-Newton method:
def quasi_newton(f, x, delta_f, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False, update_rule=BFGS_update):
    '''
    Optimizes the unconstrained problem using the quasi-Newton method
    Args:
        f: objective function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
        update_rule: update rule for the Hessian estimate
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''

    B = np.eye(x.shape[0]) # Initialize the Hessian approximation
    convergence = False
    iter = 0

    while not convergence:
        s = -B@delta_f(f, x)   # Calculate the search direction
        s = s / np.linalg.norm(s) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the step size
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter

         # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3

        B = B + update_rule(B, x_new - x, delta_f(f, x_new) - delta_f(f, x)) # Update the Hessian approximation
        x = x_new # Update x

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new

In [177]:
# Constrained optimization methods:

def calc_lagrangian(f, con, x, lambda_, mu):
    '''
    Calculates the lagrangian of the function
    Args:
        f: objective function
        con: constraint function
        x: vector of variables
        lambda_: Lagrange equality multipliers
        mu: Lagrange inequality multipliers
    Returns:
        lagrangian: lagrangian of the function
    '''
    g, h = con(x)   # Calculate the inequality and equality constraints
    
    return f(x) + lambda_@h + mu@g   # Calculate the lagrangian


# Augmented Lagrangian method:
def calc_augmented_lagrangian(f, con, x, lambda_, mu, rho):
    '''
    Calculates the augmented Lagrangian of the function
    Args:
        f: objective function
        con: constraint function
        x: vector of variables
        lambda_: Lagrange equality multipliers
        mu: Lagrange inequality multipliers
        rho: augmented Lagrangian penalty parameter
    Returns:
        augmented_lagrangian: augmented Lagrangian of the function
    '''

    g, h = con(x) # Calculate the inequality and equality constraints
    con_sum = 0
    for i in range(g.shape[0]):
        con_sum += max(0, g[i])**2
    for i in range(h.shape[0]):
        con_sum += h[i]**2
    
    return calc_lagrangian(f, con, x, lambda_, mu) +  rho * con_sum # Calculate the augmented Lagrangian function


def augmented_lagrangian(f, con, x, delta_f, rho=1, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False):
    '''
    Optimizes the constrained problem using the augmented Lagrangian method
    Args:
        f: objective function
        con: constraint function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        rho: augmented Lagrangian penalty parameter
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''
    
    g, h = con(x) # Calculate the initial constraint values
    mu0 = np.zeros(g.shape[0]).T # Initialize the inequality Lagrangian multipliers
    lambda_0 = np.zeros(h.shape[0]).T # Initialize the equality Lagrangian multipliers
    convergence = False
    iter = 0

    while not convergence:
        x_new = quasi_newton(lambda x: calc_augmented_lagrangian(f, con, x, lambda_0, mu0, rho), \
            x, delta_f, delta_f_tol, delta_x_tol, max_iter=100, full_output=False) # Optimize the augmented Lagrangian function
        g, h = con(x_new) # Calculate the constraint values
        mu_new = mu0 + rho*g # Calculate the new inequality Lagrangian multipliers
        # Check for negative inequality multipliers:
        for i in range(len(mu_new)):
            if mu_new[i] < 0:
                mu_new[i] = 0
        lambda_new = lambda_0 + rho*h # Calculate the new equality Lagrangian multipliers
        delta_L_new = delta_f(lambda x: calc_lagrangian(f, con, x, lambda_new, mu_new), x_new) # Calculate the new Lagrangian gradient
        iter += 1 # Increment the iteration counter
        
        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3
        
        x = x_new # Update x
        mu0 = mu_new # Update the Lagrangian multipliers
        lambda_0 = lambda_new # Update the Lagrangian multipliers

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    return x_new

In [178]:
# Test objective function
def f(x):
    return ( ( 1-x[0] )**2 + 100* ( x[1]-x[0]**2) **2)

# Test constraint function
def con(x):
    g = np.array([])
    h = np.array([])
    return g, h

In [179]:
#Nelder Mead Simplex Optimiser (f= Objective Function, x0= Scaled initial vector)
#http://www.scholarpedia.org/article/Nelder-Mead_algorithm




def Simplexsolver(func,x0,max_iter=10000,pert=0.25):
    iter=0
    function_array=np.zeros( ( x0.shape[0]+1,x0.shape[0]+1) )  
    i=0
    function_array[0,0]=func(x0)
    function_array[1:,0]=x0[:,0]

    print(func(x0))
    print(function_array.shape)
    while i < (len(function_array[1])-1):
      xtemp=x0
      xtemp[i]+=pert
      function_array[0,i+1]=func(xtemp)
      function_array[1:,i+1]=xtemp[:,0]
      i +=1
    
 #Parameters for simplex reflect,extend, shrink,etc(using commonly used values)

    alpha_simp=1;
    beta_simp=0.5;
    gamma_simp=2;
    delta_simp=0.5;
    
    previous_objective= 10000

    convergence=False
    while convergence==False and iter<=max_iter:
        iter +=1   
        shrink=0
       # if function_array[0,-1]-previous_objective<= 0.001:
       #  convergence=True
       # continue

       # print(function_array) 
       # print(function_array[0, :].argsort())
        function_array = function_array[:,function_array[0,:].argsort()]
        
        #print(function_array)
        previous_objective==function_array[0,-1]
        # Finding Centroid     
        c=np.zeros((x0.shape[0]))
        i=0
        while i < len(function_array[1]-1):
           c = c+function_array[1:,i]
           #print(c)
           i +=1
      
        c = c/(len(function_array[1])-1)
        #print(len(function_array[1]))
        #print(c)
        # Compute New Reflection Point 
        xr= c + alpha_simp * (c - function_array[1:,-1])
        print(c)
        print(xr)
        reflect_funcval = func(xr)
        print(reflect_funcval)
        print(function_array)

        if reflect_funcval < function_array[0,-2] and reflect_funcval >= function_array[0,0]:
          function_array[0,-1]= reflect_funcval
          function_array[1:,-1]= xr[:]
          print("reflected")
          continue
         

        if reflect_funcval  < function_array[0,0]:
             #Expand Simplex
             xe=c+gamma_si
             mp*(xr-c)
             expansion_funcval=func(xe)
             if expansion_funcval< reflect_funcval :
                function_array[0,-1]= expansion_funcval
                function_array[1:,-1]= xe[:]
                print("expanded")
                continue

             else:
                 function_array[0,-1]= reflect_funcval 
                 function_array[1:,-1]= xr[:] 
                 print("reflected")
                 continue

        if reflect_funcval  >= function_array[0,-2] :
                #Contract Simplex
                if reflect_funcval < function_array[0,-1]  :
                  xc= c + beta_simp * ( xr - c )
                  contract_funcval= func(xc)
                  if contract_funcval < reflect_funcval:
                     function_array[0,-1]= contract_funcval 
                     function_array[1:,-1]= xc[:]
                     print("Contracted")
                     continue
                  else:
                     shrink=1

                elif reflect_funcval>=function_array[0,-1] :   
                   xc= c + beta_simp * ( function_array[1:,-1] - c )
                   contract_funcval= func(xc)
                   if(contract_funcval <function_array[0,-1]):
                     function_array[0,-1]= contract_funcval 
                     function_array[1:,-1]= xc[:]
                     print("Contracted")
                     continue
                   else:
                    shrink=1  
        if shrink==1:     
            print("Shrunk")
            j=0
            print(function_array.shape[0])
            #Shrink Transformation, shrink each lattice towards the lowest value.
            while(j<function_array.shape[0]-1):
               function_array[1:,j+1]=function_array[1:,0] + delta_simp * (function_array[1:,j+1]-function_array[1:,0])
               function_array[0,j+1]=func(function_array[1:,j])
               j+=1

    return iter,function_array,function_array[0,-1]

In [180]:
x = np.array([[0.8, 0.8]], dtype=float).T # Initialize the design variable

f_unc = constrained_to_unconstrained(f, con, x) # Convert the constrained function to an unconstrained function

print(Simplexsolver(f_unc,x))

opt.minimize(f_unc, x, args=(), method='Nelder-Mead')

[2.6]
(3, 3)
[1.45  1.325]
[1.85 1.85]
247.99812500000027
[[0.278125 2.6      9.153125]
 [1.05     0.8      1.05    ]
 [1.05     0.8      0.8     ]]
Shrunk
3
[1.5125 1.45  ]
[1.975 1.975]
371.7537890625003
[[0.278125   0.278125   0.48691406]
 [1.05       0.925      1.05      ]
 [1.05       0.925      0.925     ]]
Shrunk
3
[1.54375 1.5125 ]
[2.1    2.0375]
564.0856250000007
[[0.01539307 0.278125   0.278125  ]
 [1.05       1.05       0.9875    ]
 [0.9875     1.05       0.9875    ]]
Shrunk
3
[1.559375 1.496875]
[2.06875 1.975  ]
532.3186793518074
[[0.01539307 0.70390625 1.325     ]
 [1.05       1.01875    1.05      ]
 [0.9875     0.9875     1.01875   ]]
Shrunk
3
[1.5671875 1.4890625]
[2.1      1.990625]
586.5475390625007
[[0.01539307 0.68067918 1.325     ]
 [1.05       1.05       1.034375  ]
 [0.9875     1.003125   0.9875    ]]
Shrunk
3
[1.57109375 1.48515625]
[2.0921875 1.975    ]
578.272676001192
[[0.01539307 1.15141602 1.325     ]
 [1.05       1.0421875  1.05      ]
 [0.9875     0.9875

 final_simplex: (array([[1.00001544, 1.00003065],
       [0.99999067, 0.9999826 ],
       [0.99997926, 0.99995717]]), array([2.43816670e-10, 2.46521910e-10, 6.16283605e-10]))
           fun: 2.438166698817486e-10
       message: 'Optimization terminated successfully.'
          nfev: 69
           nit: 35
        status: 0
       success: True
             x: array([1.00001544, 1.00003065])